In [1]:
# Importing the Pandas library and loading movie data
import pandas as pd
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# Loading movie ratings data
ratings=pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings.shape

(100836, 4)

In [5]:
movies.shape

(9742, 3)

In [6]:
# Creating a user-movie matrix for collaborative filtering
movies_users=ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0)

In [25]:
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Creating a sparse matrix for movie-user ratings
from scipy.sparse import csr_matrix
matrix_movies=csr_matrix(movies_users.values)
matrix_movies

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [8]:
# Building a Nearest Neighbors model for movie recommendations
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model.fit(matrix_movies)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [9]:
# Movie recommendation function using FuzzyWuzzy and Nearest Neighbors
from fuzzywuzzy import process
def recommender(movie_name,data,n):
    idx=process.extractOne(movie_name,movies['title'])[2]
    print('Movie Selected: ',movies['title'][idx],'Index: ',idx)
    print('Searching for Recommendations...')
    distance,indices=model.kneighbors(data[idx],n_neighbors=n)
    print(distance,indices)

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
# Generating movie recommendations for 'Toy Story'
recommender('toy story',matrix_movies,10)

Movie Selected:  Toy Story (1995) Index:  0
Searching for Recommendations...
[[0.         0.42739874 0.4343632  0.43573831 0.44261183 0.45290409
  0.45885465 0.4589107  0.46108723 0.46583124]] [[   0 2353  418  615  224  314  322  910  546  963]]
